# This is to fit different FOOOF models to iEEG data and save the parameters

In [66]:
# basic
import numpy as np
import os
from pathlib import Path
import pickle

# electrophys
import mne

# specparam
from fooof import FOOOF
from fooof import FOOOFGroup
from fooof.utils.params import compute_knee_frequency

# stop warnings
import warnings
warnings.filterwarnings("ignore")

## Set paths

In [73]:
# Set data path
data_path = Path('/home/b1044271/Columbia/iEEG')
# Set result path
result_path = Path('/home/b1044271/Columbia/Results/2025')
# Set dataset
dataset = 'Wakefulness_AllRegions'

# Get list of files, which correspond to regions
regions = np.sort(os.listdir(data_path / dataset))

## Set model fitting specs

In [74]:
f_min = 1
f_max = 45
t     = 10
fs    = 200

SETTINGS_W  = { 'method' : 'welch', 'average' : 'mean', 'fmin' :f_min , 'fmax':f_max, 'n_fft': fs*t, 'n_overlap': fs*t*0.5}

SETTINGS_F1={'max_n_peaks':8, 'aperiodic_mode':'knee'}
SETTINGS_F2={'max_n_peaks':8, 'aperiodic_mode':'fixed'}


## Dicts

In [75]:
# Define parameter categories
conditions = ["Wakefulness", "N2", "N3", "REM"]
metrics = ["Kn", "Exp", "Exp2", "R2K", "R2N", "EK", "EN", "PK", "PN"]

# Initialize storage dictionaries
param_data = {cond: {metric: np.zeros([len(regions), 200]) * np.nan for metric in metrics} for cond in conditions}

# Looping through epochs of each stage

In [76]:
# store PSD results
PSDs_W = np.zeros([len(regions), 441]) * np.nan
PSDs_N2 = np.zeros([len(regions), 441]) * np.nan
PSDs_N3 = np.zeros([len(regions), 441]) * np.nan
PSDs_R = np.zeros([len(regions), 441]) * np.nan


for condition in conditions:
    dataset = f"{condition}_AllRegions"
    regions = np.sort(os.listdir(data_path / dataset))
    i = 0
    for C, V in enumerate(regions):
        
        EEG = mne.io.read_raw_edf(data_path / dataset / V)
        EEG_psd = EEG.compute_psd(**SETTINGS_W)
        
        if condition == "Wakefulness":
            PSDs_W[i, 0:len(EEG_psd._freqs)] = np.mean(EEG_psd._data, 0)
        elif condition == "N2":
            PSDs_N2[i, 0:len(EEG_psd._freqs)] = np.mean(EEG_psd._data, 0)
        elif condition == "N3":
            PSDs_N3[i, 0:len(EEG_psd._freqs)] = np.mean(EEG_psd._data, 0)
        elif condition == "REM":
            PSDs_R[i, 0:len(EEG_psd._freqs)] = np.mean(EEG_psd._data, 0)
        
        fm1 = FOOOFGroup(**SETTINGS_F1)
        fm1.fit(EEG_psd._freqs, EEG_psd._data, [EEG_psd._freqs[0], EEG_psd._freqs[-1]])
        
        fm2 = FOOOFGroup(**SETTINGS_F2)
        fm2.fit(EEG_psd._freqs, EEG_psd._data, [EEG_psd._freqs[0], EEG_psd._freqs[-1]])
        
        param_data[condition]["Kn"][i, 0:len(EEG_psd._data)] = compute_knee_frequency(fm1.get_params('aperiodic_params', 'knee'), fm1.get_params('aperiodic_params', 'exponent'))
        param_data[condition]["Exp"][i, 0:len(EEG_psd._data)] = fm1.get_params('aperiodic_params', 'exponent')
        param_data[condition]["Exp2"][i, 0:len(EEG_psd._data)] = fm2.get_params('aperiodic_params', 'exponent')
        
        param_data[condition]["R2K"][i, 0:len(EEG_psd._data)] = fm1.get_params('r_squared')
        param_data[condition]["R2N"][i, 0:len(EEG_psd._data)] = fm2.get_params('r_squared')
        
        param_data[condition]["EK"][i, 0:len(EEG_psd._data)] = fm1.get_params('error')
        param_data[condition]["EN"][i, 0:len(EEG_psd._data)] = fm2.get_params('error')
        
        # Extract the number of peaks for each epoch
        peak_params1 = fm1.get_params('peak_params')
        peak_params2 = fm2.get_params('peak_params')
        
        for epoch_idx in range(len(EEG_psd._data)):
            num_peaks1 = sum(peak[-1] == epoch_idx for peak in peak_params1)  # Count peaks for each epoch
            num_peaks2 = sum(peak[-1] == epoch_idx for peak in peak_params2)  # Count peaks for each epoch
            
            param_data[condition]["PK"][i, epoch_idx] = num_peaks1
            param_data[condition]["PN"][i, epoch_idx] = num_peaks2 
    
        i += 1

Extracting EDF parameters from /home/b1044271/Columbia/iEEG/Wakefulness_AllRegions/Amygdala_W.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Effective window size : 10.000 (s)
Running FOOOFGroup across 6 power spectra.
Running FOOOFGroup across 6 power spectra.
Extracting EDF parameters from /home/b1044271/Columbia/iEEG/Wakefulness_AllRegions/Angular gyrus_W.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Effective window size : 10.000 (s)
Running FOOOFGroup across 53 power spectra.
Running FOOOFGroup across 53 power spectra.
Extracting EDF parameters from /home/b1044271/Columbia/iEEG/Wakefulness_AllRegions/Anterior cingulate_W.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Effective window size : 10.000 (s)
Running FOOOFGroup across 31 power spectra.
Running FOOOFGroup across 31 power spectra.
Extracting EDF parameters from /home/b1044271/Columbia/iEEG/Wakefu

## Save

In [78]:
save_path = Path(result_path) / 'iEEG_results10.pkl'
with open(save_path, 'wb') as f:
    pickle.dump({'PSDs': {'Wakefulness': PSDs_W, 'N2': PSDs_N2, 'N3': PSDs_N3, 'REM': PSDs_R}, 'param_data': param_data}, f)

# save freqs
np.save(Path(result_path /'iEEG_Freqs10.npy'),EEG_psd.freqs)